# vector db 활용 학습
- 주제: @@@ 감성 분석 or 사용자 맞춤 데이터 출력
1. 적절한 크기의 데이터셋을 구한다.
2. 한국어 임베딩에 적절한 모델을 찾는다.
3. Chroma DB, FAISS, Pinecone 중 원하는 벡터 DB를 이용한다.

In [4]:
# import pandas as pd
# import numpy as np

# df = pd.read_csv('./data/train.tsv', sep='\t')

# print(f"원본 데이터 크기: {len(df)}")

# sample_10k = df.sample(n=10000, random_state=42)  # 재현성을 위한 random_state
# sample_1k = df.sample(n=1000, random_state=42)

# # CSV 파일로 저장
# sample_10k.to_csv('./data/sentiment_10k_sample.csv', index=False)
# sample_1k.to_csv('./data/train_1k.csv', index=False)

# print(f"10,000개 샘플 크기: {len(sample_10k)}")
# print(f"1,000개 샘플 크기: {len(sample_1k)}")

In [5]:
# import pandas as pd
# import numpy as np

# df = pd.read_csv('./data/train_1k.csv')

# df = df[['comments','hate']]

# hate_mapping = {
#     'none': 0,      # 혐오 표현 없음
#     'offensive': 1, # 공격적인 표현 (약한 혐오)
#     'hate': 2       # 명백한 혐오 표현
# }

# # 매핑 적용
# df['hate'] = df['hate'].map(hate_mapping)

# df.to_csv('./data/hate.csv', index=False)

In [17]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# 데이터 로드
df = pd.read_csv('./data/hate.csv')
print(f"데이터셋 크기: {len(df)} 행")
print(f"컬럼: {df.columns.tolist()}")

# 임베딩 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# 전체 데이터셋 임베딩 계산 및 저장
print("데이터셋 임베딩 생성 중...")
all_embeddings = []
for text in df['comments']:
    embedding = model.encode(text)
    all_embeddings.append(embedding)

all_embeddings = np.array(all_embeddings)
print(f"임베딩 생성 완료: {all_embeddings.shape}")

데이터셋 크기: 1000 행
컬럼: ['comments', 'hate']
데이터셋 임베딩 생성 중...
임베딩 생성 완료: (1000, 384)


In [18]:
from sklearn.metrics.pairwise import cosine_similarity


# 공격성 분석 함수
def analyze_aggression(query_text, top_k=10):
    # 텍스트 기반 공격성 분석
    offensive_word_scores = {
        # 높은 수준의 혐오 표현 (8-10점)
        "씨발": 10, "시발": 10, "병신": 9, "지랄": 9, "개새끼": 10, "좆": 9, 
        "씹": 8, "년": 8, "놈": 7, "닥쳐": 8, "꺼져": 8,
        
        # 중간 수준의 공격적 표현 (5-7점)
        "새끼": 7, "미친": 6, "죽어": 7, "죽여": 9, "바보": 5, "멍청이": 6,
        "꺼지": 6, "뒤져": 7, "아가리": 7, "쓰레기": 6, "쓰렉": 6,
        
        # 낮은 수준의 부정적 표현 (2-4점)
        "짜증": 3, "싫어": 2, "나쁜": 3, "못생긴": 3, "멍청한": 4, "못된": 4
    }
    
    # 텍스트에서 공격적 단어 찾기
    text_lower = query_text.lower()
    found_offensive_words = []
    
    for word, score in offensive_word_scores.items():
        if word in text_lower:
            found_offensive_words.append((word, score))
    
    # 최대 공격성 점수 계산
    text_score = 0
    if found_offensive_words:
        text_score = max(score for _, score in found_offensive_words)
    
    # 쿼리 임베딩 계산
    query_embed = model.encode(query_text)
    
    # 모든 댓글과의 유사도 계산
    similarities = cosine_similarity([query_embed], all_embeddings)[0]
    
    # 유사도 내림차순으로 정렬된 인덱스
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    # 유사한 댓글 정보 수집
    similar_comments = []
    for idx in top_indices:
        comment_text = df.iloc[idx]['comments']
        hate_value = df.iloc[idx]['hate']
        similarity = similarities[idx]
        
        # 댓글 텍스트 분석으로 점수 추정
        comment_score = 0
        for word, score in offensive_word_scores.items():
            if word in comment_text.lower():
                comment_score = max(comment_score, score)
        
        # 'hate' 값을 0-10 스케일로 변환
        hate_score = 0
        if hate_value == 1:
            hate_score = 5
        elif hate_value == 2:
            hate_score = 10
            
        # 최종 점수는 텍스트 분석과 레이블 중 높은 값
        final_score = max(comment_score, hate_score)
        
        similar_comments.append({
            'text': comment_text,
            'hate_value': hate_value,
            'display_score': final_score,
            'text_score': comment_score,
            'label_score': hate_score,
            'similarity_score': similarity
        })
    
    # 공격성 레벨 결정
    if text_score < 3:
        aggression_level = "낮음 (중립적 표현)"
    elif text_score < 7:
        aggression_level = "중간 (약간 공격적)"
    else:
        aggression_level = "높음 (매우 공격적/혐오 표현)"
    
    return {
        'normalized_score': text_score,
        'aggression_level': aggression_level,
        'similar_comments': similar_comments,
        'found_offensive_words': found_offensive_words
    }

# 메인 로직
print("\n===== 공격성 분석 시스템 =====")
print("텍스트를 입력하면 공격성 정도를 0-10점 척도로 분석합니다.")

while True:
    query_text = input("\n분석할 텍스트를 입력하세요 (종료하려면 'q' 입력): ")
    
    if query_text.lower() == 'q':
        break
    
    # 분석 실행
    result = analyze_aggression(query_text)
    
    # 결과 출력
    print("\n===== 공격성 분석 결과 =====")
    print(f"입력 텍스트: {query_text}")
    print(f"공격성 점수: {result['normalized_score']:.1f}/10")
    print(f"공격성 수준: {result['aggression_level']}")
    
    # 발견된 공격적 단어 출력
    if result['found_offensive_words']:
        print("\n발견된 공격적 표현:")
        for word, score in result['found_offensive_words']:
            print(f"- '{word}' (점수: {score})")
    
    print("\n----- 유사한 댓글 (참고) -----")
    for i, comment in enumerate(result['similar_comments'][:5], 1):
        print(f"{i}. [원본: {comment['hate_value']}, 점수: {comment['display_score']}/10] {comment['text'][:100]}...")
        print(f"   (텍스트 점수: {comment['text_score']}, 레이블 점수: {comment['label_score']}, 유사도: {comment['similarity_score']:.4f})")


===== 공격성 분석 시스템 =====
텍스트를 입력하면 공격성 정도를 0-10점 척도로 분석합니다.

===== 공격성 분석 결과 =====
입력 텍스트: 이것도 못하냐 ㅋㅋ
공격성 점수: 0.0/10
공격성 수준: 낮음 (중립적 표현)

----- 유사한 댓글 (참고) -----
1. [원본: 2, 점수: 10/10] 양세형 빨갱이인줄...
   (텍스트 점수: 0, 레이블 점수: 10, 유사도: 0.9579)
2. [원본: 0, 점수: 0/10] 꿀잼 ㅠㅠ...
   (텍스트 점수: 0, 레이블 점수: 0, 유사도: 0.9579)
3. [원본: 1, 점수: 5/10] 똥싸고 자빠졌네...
   (텍스트 점수: 0, 레이블 점수: 5, 유사도: 0.9579)
4. [원본: 0, 점수: 0/10] 넘재밌었음강문영씨화이팅 김광규씨화이팅최성국시나오디마...
   (텍스트 점수: 0, 레이블 점수: 0, 유사도: 0.9579)
5. [원본: 2, 점수: 10/10] 얘가뭔죄고 맷퇘지들아ㅋㅋㅋ...
   (텍스트 점수: 0, 레이블 점수: 10, 유사도: 0.9579)

===== 공격성 분석 결과 =====
입력 텍스트: 쓰래기야
공격성 점수: 0.0/10
공격성 수준: 낮음 (중립적 표현)

----- 유사한 댓글 (참고) -----
1. [원본: 2, 점수: 10/10] 양세형 빨갱이인줄...
   (텍스트 점수: 0, 레이블 점수: 10, 유사도: 0.9021)
2. [원본: 0, 점수: 0/10] 넘재밌었음강문영씨화이팅 김광규씨화이팅최성국시나오디마...
   (텍스트 점수: 0, 레이블 점수: 0, 유사도: 0.9021)
3. [원본: 2, 점수: 10/10] 얘가뭔죄고 맷퇘지들아ㅋㅋㅋ...
   (텍스트 점수: 0, 레이블 점수: 10, 유사도: 0.9021)
4. [원본: 1, 점수: 5/10] 똥싸고 자빠졌네...
   (텍스트 점수: 0, 레이블 점수: 5, 유사도: 0.9021)
5. [원본: 0, 점수: 0/10] 꿀잼 ㅠㅠ...
   (텍스